Here we try to reproduce the assembly documentation from this [best practices example](https://github.com/SynBioDex/SBOL-examples/blob/main/SBOL/best-practices/BP011/examples.md)



In [111]:
import sbol3
import tyto

In [112]:
sbol3.set_namespace('http://sbolstandard.org/testfiles')

# Create sequences
plasmid_seq = sbol3.Sequence('plasmid_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)
pcr_prod_seq = sbol3.Sequence('pcr_prod_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)
final_prod_seq = sbol3.Sequence('final_prod_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)

restricted_plasmid_seq = sbol3.Sequence('restricted_plasmid_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)
restricted_pcr_prod_seq = sbol3.Sequence('restricted_pcr_prod_seq' , elements='aaaaaaaaaa', encoding=sbol3.IUPAC_DNA_ENCODING)

salI_plasmid = sbol3.SequenceFeature(sbol3.Range(plasmid_seq, 37, 42), roles=[tyto.SO.restriction_enzyme_recognition_site], name='salI_on_plasmid')
ascI_plasmid = sbol3.SequenceFeature(sbol3.Range(plasmid_seq, 62, 69), roles=[tyto.SO.restriction_enzyme_recognition_site], name='ascI_on_plasmid')

salI_pcr_prod = sbol3.SequenceFeature(sbol3.Range(pcr_prod_seq, 7, 12), roles=[tyto.SO.restriction_enzyme_recognition_site], name='salI_on_pcr_prod')
ascI_pcr_prod = sbol3.SequenceFeature(sbol3.Range(pcr_prod_seq, 2256, 2263), roles=[tyto.SO.restriction_enzyme_recognition_site], name='ascI_on_pcr_prod')

plasmid = sbol3.Component('plasmid', sbol3.SBO_DNA, features=[salI_plasmid, ascI_plasmid], sequences=[plasmid_seq], description='plasmid')
pcr_prod = sbol3.Component('pcr_prod', sbol3.SBO_DNA, features=[salI_pcr_prod, ascI_pcr_prod], sequences=[pcr_prod_seq], description='pcr_prod')
final_prod = sbol3.Component('final_prod', sbol3.SBO_DNA, sequences=[final_prod_seq], description='final_prod')
restricted_plasmid = sbol3.Component('restricted_plasmid', sbol3.SBO_DNA, sequences=[restricted_plasmid_seq], description='restricted_plasmid')
restricted_pcr_prod = sbol3.Component('restricted_pcr_prod', sbol3.SBO_DNA, sequences=[restricted_pcr_prod_seq], description='restricted_pcr_prod')

plasmid_subcomp = sbol3.SubComponent(plasmid)
pcr_prod_subcomp = sbol3.SubComponent(pcr_prod)
final_prod_subcomp = sbol3.SubComponent(final_prod)
restricted_plasmid_subcomp = sbol3.SubComponent(restricted_plasmid)
restricted_pcr_prod_subcomp = sbol3.SubComponent(restricted_pcr_prod)

final_prod.features.extend([plasmid_subcomp, pcr_prod_subcomp, final_prod_subcomp, restricted_plasmid_subcomp, restricted_pcr_prod_subcomp])


In [113]:
# Digestion interactions

salI_subcomp = sbol3.SubComponent('http://purl.obolibrary.org/obo/PR_Q53608', description='EcoRI restriction enzyme')
ascI_subcomp = sbol3.SubComponent('http://purl.obolibrary.org/obo/PR_E3VXA3', description='XbaI restriction enzyme')

final_prod.features.extend([salI_subcomp, ascI_subcomp])

salI_participation_plasmid = sbol3.Participation(tyto.SBO.modifier, salI_subcomp)
ascI_participation_plasmid = sbol3.Participation(tyto.SBO.modifier, ascI_subcomp)
plasmid_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, plasmid_subcomp)
restricted_plasmid_product_participation = sbol3.Participation(tyto.SBO.product, restricted_plasmid_subcomp)


plasmid_digestion = sbol3.Interaction(tyto.SBO.cleavage, name='plasmid_digestion', participations=[
    plasmid_reactant_participation,
    restricted_plasmid_product_participation,
    salI_participation_plasmid,
    ascI_participation_plasmid
])

plasmid_digestion_component = sbol3.Component('plasmid_digestion_component', sbol3.SBOL_COMPONENT, interactions=[plasmid_digestion])

# Add a constraint components references to cutsites of interest left_cut --precedes--> right_cut (see figure backbone)
# this sequence feature is in a precedes relationship, or after

salI_participation_pcr_prod = sbol3.Participation(tyto.SBO.modifier, salI_subcomp)
ascI_participation_pcr_prod = sbol3.Participation(tyto.SBO.modifier, ascI_subcomp)
pcr_prod_reaction_participation = sbol3.Participation(sbol3.SBO_REACTANT, pcr_prod_subcomp)
restricted_pcr_prod_product_participation = sbol3.Participation(tyto.SBO.product, restricted_pcr_prod_subcomp)

pcr_prod_digestion = sbol3.Interaction(tyto.SBO.cleavage, name='pcr_prod_digestion', participations=[
    pcr_prod_reaction_participation,
    restricted_pcr_prod_product_participation,
    salI_participation_pcr_prod,
    ascI_participation_pcr_prod
])

pcr_prod_digestion_component = sbol3.Component('pcr_prod_digestion_component', sbol3.SBOL_COMPONENT, interactions=[pcr_prod_digestion])

In [114]:
# Ligation interaction
ligase_subcomp = sbol3.SubComponent('http://purl.obolibrary.org/obo/blah', description='T4 DNA ligase')
pcr_prod.features.append(ligase_subcomp)

ligase_participation = sbol3.Participation(tyto.SBO.modifier, ligase_subcomp)
restricted_plasmid_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, restricted_plasmid_subcomp)
restricted_pcr_prod_reactant_participation = sbol3.Participation(sbol3.SBO_REACTANT, restricted_pcr_prod_subcomp)
final_prod_product_participation = sbol3.Participation(tyto.SBO.product, final_prod_subcomp)

ligation = sbol3.Interaction(tyto.SBO.conversion, name='ligation', participations=[
    restricted_plasmid_reactant_participation,
    restricted_pcr_prod_reactant_participation,
    final_prod_product_participation,
    ligase_participation
])

ligation_component = sbol3.Component('ligation_component', sbol3.SBOL_COMPONENT, interactions=[ligation])




In [115]:
# Prepare Document
doc = sbol3.Document()
doc.add(plasmid)
doc.add(pcr_prod)
doc.add(final_prod)
doc.add(plasmid_digestion_component)
doc.add(pcr_prod_digestion_component)
doc.add(ligation_component)
doc.add(restricted_plasmid)
doc.add(restricted_pcr_prod)

doc.write('example_combine.ttl')
doc.write('example_combine.xml')

!bash make_graph.sh

showing instances: True
showing classes: False
showing namespaces: True
using rdf format: turtle
graphic file created: example_combine.ttl.svg
graphic was created with output file name: example_combine.ttl.svg
